In [ ]:
# ==========================================================
# Maestría en Ciencia y Análisis de Datos
# Universidad Mayor de San Andrés
# ----------------------------------------------------------
#   Modelos lineales y modelos lineales generalizados
# ----------------------------------------------------------
#        Rolando Gonzales Martinez, Julio 2024
# ==========================================================
# Modelo lineal multivariante: evaluacion de modelos 
# Linealidad, normalidad, homocedasticidad y correlacion de los errores 

# Importando librerias:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms


# Cargar el conjunto de datos:
url = 'https://raw.githubusercontent.com/rogon666/UMSA/main/MLMLG/datos/salarios.csv'

# Cargar los datos en un DataFrame
datos = pd.read_csv(url)

# Mostrar las primeras filas del DataFrame
print(datos.head())

In [ ]:
# Definir las variables independientes y dependientes
X = datos[['educacion', 'edad', 'educacion_posgrado']]
y = datos['salario']

# Añadir una constante a las variables independientes
X = sm.add_constant(X)

# Ajustar el modelo de regresión lineal
modelo_OLS = sm.OLS(y, X).fit()

# Resumen del modelo
print(modelo_OLS.summary())

In [ ]:
# Ajustando el modelo de regresion utilizando una formula:
resultados = smf.ols("salario ~ educacion + edad + educacion_posgrado", data=datos).fit()

# Resultados
print(resultados.summary())

In [ ]:
# Test de linealidad: Harvey-Collier
test = sms.linear_harvey_collier(resultados)
print(f'Estadígrafo HC: {test[0]}')
print(f'p-value HC ~ t(n-k-1): {test[1]}')

In [ ]:
# Test de normalidad de residuos: Jarque-Bera
test = sms.jarque_bera(resultados.resid)
print(f'Estadígrafo Jarque-Bera (JB): {test[0]}')
print(f'p-value JB ~ chi^2(2): {test[1]}')
print(f'Sesgo: {test[2]}')
print(f'Curtosis: {test[3]}')

In [ ]:
# Test de normalidad de residuos: Omnibus de D’Agostino and Pearson
test = sms.omni_normtest(resultados.resid)
print(f'Estadígrafo Omni: {test[0]}')
print(f'p-value omni ~ chi^2(2): {test[1]}')
# D’Agostino, R. B. (1971), “An omnibus test of normality for moderate and large sample size”, Biometrika, 58, 341-348
# D’Agostino, R. and Pearson, E. S. (1973), “Tests for departure from normality”, Biometrika, 60, 613-622

In [ ]:
# Test Homocedasticidad: Breusch–Pagan
test = sms.het_breuschpagan(resultados.resid, resultados.model.exog)
print(f'Estadígrafo LMBP: {test[0]}')
print(f'p-value LMBP ~ chi^2(k-1): {test[1]}')
print(f'Estadígrafo FBP: {test[2]}')
print(f'p-value FBP ~ F(k,n-k-1): {test[3]}')

In [ ]:
# Test Homocedasticidad: Goldfeld-Quandt
test = sms.het_goldfeldquandt(resultados.resid, resultados.model.exog)
print(f'Estadígrafo FGQ: {test[0]}')
print(f'p-value FGQ ~ F(n1-k,n2-k): {test[1]}')

In [ ]:
# Test Homocedasticidad: White
from statsmodels.stats.diagnostic import het_white
white_test = het_white(resultados.resid, resultados.model.exog)
print(f'Estadístico LMW: {white_test[0]}')
print(f'p-value LMW ~ chi^2(k-1): {white_test[1]}')

In [ ]:
# Transformaciones de variables
resultados_logsal = smf.ols("np.log(salario) ~ educacion + edad + educacion_posgrado", data=datos).fit()

# Resultados
print(resultados_logsal.summary())

# Test Homocedasticidad: White
from statsmodels.stats.diagnostic import het_white
white_test = het_white(resultados_logsal.resid, resultados_logsal.model.exog)
print(f'Estadístico LMW: {white_test[0]}')
print(f'p-value LMW ~ chi^2(k-1): {white_test[1]}')

In [ ]:
# Estimacion con minimos cuadrados ponderados (WLS):
import matplotlib.pyplot as plt

# Calcular los residuos del modelo OLS
residuos = modelo_OLS.resid

# Calcular los pesos inversamente proporcionales a los residuos al cuadrado (o a otra medida de varianza)
pesos = 1 / (residuos**2)

# Ajustar el modelo de regresión ponderada usando WLS
wls_model = sm.WLS(y, X, weights=pesos).fit()

# Mostrar el resumen del modelo OLS
print("Resumen del modelo OLS:")
print(modelo_OLS.summary())

# Mostrar el resumen del modelo WLS
print("\nResumen del modelo WLS:")
print(wls_model.summary())

# Graficar los residuos para observar la heterocedasticidad
plt.scatter(modelo_OLS.fittedvalues, residuos)
plt.xlabel('Valores Ajustados')
plt.ylabel('Residuos')
plt.title('Gráfico de Residuos (OLS)')
plt.axhline(y=0, color='r', linestyle='--')
plt.show()

# Graficar los residuos ponderados para observar la heterocedasticidad corregida
plt.scatter(wls_model.fittedvalues, wls_model.resid)
plt.xlabel('Valores Ajustados')
plt.ylabel('Residuos Ponderados')
plt.title('Gráfico de Residuos Ponderados (WLS)')
plt.axhline(y=0, color='r', linestyle='--')
plt.show()

In [ ]:
# Calcular errores estándar robustos usando la matriz de covarianza de tipo HC3
robust_cov = modelo_OLS.get_robustcov_results(cov_type='HC0')

# Mostrar los resultados con errores estándar robustos
print(robust_cov.summary())

In [ ]:
# Correlacion de los errores: Test de Durbin-Watson
from statsmodels.stats.stattools import durbin_watson
from scipy.stats import norm
dw_stat = durbin_watson(modelo_OLS.resid)
print(f'Estadística de Durbin-Watson: {dw_stat}')
# Interpretación basada en valores críticos (asumiendo un nivel de significancia de 0.05)
# Nota: Los valores críticos dL y dU dependen de n y k
dL = 1.738 # Valor crítico inferior para n = 177 y k = 3 (excluyendo la constante)
dU = 1.799  # Valor crítico superior para n = 177 y k = 3 (excluyendo la constante)
if dw_stat < dL:
    print("Evidencia de autocorrelación positiva de primer orden")
elif dL <= dw_stat <= dU:
    print("Prueba inconclusa.")
else:
    print("No hay evidencia de autocorrelación positiva.")
# Nota: el estadigrafo DW es sesgado (subestima autocorrelacion) en el contexto de AR(I)MA
# Calcular la estadística H de Durbin-Watson
residuos = modelo_OLS.resid
rho_hat = np.corrcoef(residuos[:-1], residuos[1:])[0, 1]
n = len(modelo_OLS.resid)
k = 5  # Número de variables explicativas excluyendo la constante
h_stat = (1 - dw_stat / 2) * np.sqrt(n / (1 - n * (rho_hat**2)))
print(f'Estadígrafo H de Durbin-Watson: {h_stat}')
# Evaluar la estadística H usando la distribución normal estándar
alpha = 0.05 # significancia de 5%
z_critical = norm.ppf(1 - alpha / 2)  # Valor crítico para un nivel de significancia de 5%
print(f'Valor crítico z para H: {z_critical}')

In [ ]:
# Correlacion de los errores: Test Breusch-Godfrey
from statsmodels.stats.diagnostic import acorr_breusch_godfrey
bg_test = acorr_breusch_godfrey(modelo_OLS, nlags=5)
# Resultados del test de Breusch-Godfrey
print(f'Estadístico LM: {bg_test[0]}')
print(f'Valor p (LM): {bg_test[1]}')
print(f'Estadístico F: {bg_test[2]}')
print(f'Valor p (F): {bg_test[3]}')
# Interpretación de los resultados
alpha = 0.05
if bg_test[1] < alpha:
    print("Se rechaza la hipótesis nula de no correlación de los errores. \nExiste evidencia de correlación de los errores.")
else:
    print("No se rechaza la hipótesis nula de no correlación de los errores. \nNo hay evidencia de correlación de los errores.")

In [ ]:
# Graficar la función de autocorrelación de los residuos
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(modelo_OLS.resid)
plt.show()

In [ ]:
# Calcular el número de rezagos usando la regla de Parzen
n = len(y)
rezagos_parzen = int(0.75 * n**(1/3))
print(f"Número de rezagos según la regla de Parzen: {rezagos_parzen}")

In [ ]:
# Resultados con la matriz varianza-covarianza Newey-West:
neweywest_cov = modelo_OLS.get_robustcov_results(cov_type='HAC', 
                                                 maxlags=4, 
                                                 use_correction=True)
print(neweywest_cov.summary())

In [ ]:
# Analisis de observaciones influyentes:
influencia = modelo_OLS.get_influence()
# Obtener los valores de leverage
apanlacamiento = influencia.hat_matrix_diag
# Graficar los valores de apanlacamiento (leverage)
plt.figure(figsize=(10, 6))
plt.stem(apanlacamiento)
plt.xlabel('Índice de la observación')
plt.ylabel('Leverage')
plt.title('Valores de apanlacamiento (leverage) para cada observación')
plt.grid(True)
plt.show()

In [ ]:
# Distancia de Cook
d_cook = influencia.cooks_distance[0]
# Valores mayores a 1 sugieren observaciones influyentes.
# Generar el gráfico de Distancia de Cook
umbral = 1 
plt.figure(figsize=(10, 6))
plt.stem(d_cook)
plt.axhline(y=umbral, color='r', linestyle='--', label=f'Umbral = {umbral:.2f}')
plt.xlabel('Índice de la observación')
plt.ylabel('Distancia de Cook')
plt.title('Distancia de Cook para cada observación')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Grafico de influencia:
from statsmodels.graphics.regressionplots import influence_plot
fig, ax = plt.subplots(figsize=(10, 8))
influence_plot(modelo_OLS, ax=ax)
plt.show()

In [ ]:
# Ajustar el nuevo modelo sin observaciones influyentes
obs_a_remover = 102
# Eliminar la fila especificada
# Definir las variables independientes y dependientes
X = datos[['educacion', 'edad', 'educacion_posgrado','antiguedad_ejecutivo']]
y = datos['salario']
X_ni = np.delete(X, obs_a_remover, axis=0)
y_ni = np.delete(y, obs_a_remover, axis=0)
X_ni = sm.add_constant(X_ni)
#X_ni.columns = ['constante', 'educacion', 'edad','educacion_posgrado','antiguedad_ejecutivo'] 
X_ni = pd.DataFrame(X_ni, columns=['constante','educacion', 'edad', 'educacion_posgrado', 'antiguedad_ejecutivo'])
# Resumen del nuevo modelo ajustado
modelo_ni = sm.OLS(y_ni, X_ni).fit()
print(modelo_ni.summary())
robust_cov_ni = modelo_ni.get_robustcov_results(cov_type='HC0')
neweywest_cov = modelo_ni.get_robustcov_results(cov_type='HAC', maxlags=4, use_correction=True)
print(robust_cov_ni.summary())
print(neweywest_cov.summary())
print('--------------- Correlación en los residuos -----------------------')
bg_test = acorr_breusch_godfrey(modelo_OLS, nlags=5)
print(f'Estadístico LM: {bg_test[0]}')
print(f'Valor p (LM): {bg_test[1]}')
print(f'Estadístico F: {bg_test[2]}')
print(f'Valor p (F): {bg_test[3]}')
alpha = 0.05
if bg_test[1] < alpha:
    print("Se rechaza la hipótesis nula de no correlación de los errores. \nExiste evidencia de correlación de los errores.")
else:
    print("No se rechaza la hipótesis nula de no correlación de los errores. \nNo hay evidencia de correlación de los errores.")
white_test = het_white(resultados_logsal.resid, resultados_logsal.model.exog)
print('--------------- Heterocedasticidad en los residuos -----------------------')
print(f'Estadístico LMW: {white_test[0]}')
print(f'p-value LMW ~ chi^2(k-1): {white_test[1]}')
if white_test[1] < alpha:
    print("Se rechaza la hipótesis nula de homocedasticidad. \nExiste evidencia de heterocedasticidad de los errores.")
else:
    print("No se rechaza la hipótesis nula de homocedasticidad. \nNo hay evidencia de heterocedasticidad de los errores.")
print('============================================================================')